<a href='http://www.holoviews.org'><img src="assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>08. Deploying Bokeh Apps</h2></div>

In the previous sections we discovered how to create static, interactive plots using a ``HoloMap`` and declare dynamic interactivity using ``DynamicMap`` and ``Streams`` inside a Jupyter notebook. However, frequently we want to package our visualization or dashboard for wider distribution outside the notebook. Bokeh server provides a flexible and scalable architecture to deploy complex interactive visualizations and dashboards and integrates seemlessly with HoloViews.

For a detailed background on bokeh server see [the bokeh user guide](http://bokeh.pydata.org/en/latest/docs/user_guide/server.html). In this tutorial we will discover how to deploy the visualizations we have created so far as a standalone bokeh server app and how flexibly combine HoloViews and Bokeh APIs to build highly customized apps. We will also reuse a lot of what we have learned so far, loading large, tabular datasets, applying datashader operations to them and adding linked streams to our app.

## A simple bokeh app

```python
import dask.dataframe as dd
from holoviews.operation.datashader import datashade
import holoviews as hv

renderer = hv.renderer('bokeh')

# Load data
ddf = dd.read_csv('../data/nyc_taxi.csv', usecols=['dropoff_x', 'dropoff_y']).persist()

# Declare points and datashade them
shaded = datashade(hv.Points(ddf))

# Set some plot options
app = shaded.opts(plot=dict(width=800, height=400))

# Create a bokeh document
doc = renderer.server_doc(app)
doc.title = 'HoloViews Bokeh App'
```

You will be familiar with most of the components of this app by now. We load the dropoff locations from the Taxi dataset using dask, we declare some Points, datashade them and we set some plot options. Only the top and bottom should be unfamiliar. Instead of loading the bokeh extension we get a handle on a bokeh renderer using the ``hv.renderer`` function and as a last step we obtain a bokeh document by passing the HoloViews object to the ``renderer.server_doc`` method. That's all that's required to turn a HoloViews object into a functional bokeh app!

To run this app open [a terminal](../terminals/1) and execute:

```
source activate hvtutorial
bokeh serve --show apps/server_app.py
```

Now [open the app script file](../edit/apps/server_app.py) in the inbuilt text editor.

In [ ]:
# Exercise: Modify the app to display the pickup locations and add a tilesource, then run the app with bokeh serve
# Tip: Refer to the previous notebook

## Building a bokeh app in the notebook

In a usual workflow you often want to quickly iterate over a visualization in the notebook and only deploy it when we are happy with it. We will quickly go through such a workflow. So as before we will set up our imports, load the extension and the taxi dataset:

In [ ]:
import holoviews as hv
import geoviews as gv
import dask.dataframe as dd

from holoviews.operation.datashader import datashade, aggregate, shade
from bokeh.models import WMTSTileSource

hv.extension('bokeh', logo=False)

usecols = ['tpep_pickup_datetime', 'dropoff_x', 'dropoff_y']
ddf = dd.read_csv('../data/nyc_taxi.csv', parse_dates=['tpep_pickup_datetime'], usecols=usecols)
ddf['hour'] = ddf.tpep_pickup_datetime.dt.hour
ddf = ddf.persist()

Next we define a ``Counter`` stream which we will use to select taxi trips by hour.

In [ ]:
stream = hv.streams.Counter()
points = hv.Points(ddf, kdims=['dropoff_x', 'dropoff_y'])
dmap = hv.DynamicMap(lambda counter: points.select(hour=counter%24).relabel('Hour: %s' % (counter % 24)),
                     streams=[stream])
shaded = datashade(dmap)

hv.opts('RGB [width=800, height=600, xaxis=None, yaxis=None]')

url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
wmts = gv.WMTS(WMTSTileSource(url=url))

overlay = wmts * shaded

As before we now have a fully functional bokeh app and to see what that looks like we can even display it inline using the ``renderer.app`` method using the ``show`` argument. If your Jupyter notebook server is not running on port 8888 you will also have to edit the ``websocket_origin`` to match:

In [ ]:
renderer = hv.renderer('bokeh')
server = renderer.app(overlay, show=True, websocket_origin='localhost:8888')

Bokeh server also allows us to schedule periodic events and the ``periodic`` method on a DynamicMap declaring a ``Counter`` stream is the easiest way to do that.

In [ ]:
dmap.periodic(1)

You can stop the server process by clearing the cell displaying the app.

Now let's open the [text editor](../edit/apps/periodic_app.py) again.

In [ ]:
# Exercise: Copy the example above into periodic_app.py and modify it so it can be run with bokeh serve
# Hint: Use hv.renderer and renderer.server_doc
# Note that you have to run periodic **after** creating the bokeh document

## Combining HoloViews with bokeh models

Now for a last hurrah let's put everything we have learned to good use and create a bokeh app with it. This time we will go straight to a [Python script containing the app](../edit/apps/player_app.py). If you run the app with ``bokeh serve --show ./apps/player_app.py`` from [your terminal](../terminals/1) you should see something like this:

<img src="./assets/tutorial_app.gif"></img>

The app consists of several components:

1. A datashaded plot of points letting us select the Points for each hour of the day
2. A linked ``PointerX`` stream which is used to compute a cross-section
3. A set of custom bokeh widgets linked to the hour of day stream

We have already covered 1. and 2. so we will focus on 3. and find out how easily we can combine a HoloViews plot with custom bokeh models. We will not look at the precise widgets in too much detail, instead let's have a quick look at the callback define for slider widget updates:

```python
def slider_update(attrname, old, new):
    stream.event(hour=new)
```

Whenever the slider value changes this will trigger a stream event updating our plots. The second part is how we combine HoloViews objects and bokeh models into a single layout we can display. Once again we can use the renderer to convert the HoloViews object into something we can display with bokeh:

```python
renderer = hv.renderer('bokeh')
plot = renderer.get_plot(hvobj, doc=curdoc())
```

The ``plot`` instance here has a ``state`` attribute that represents the actual bokeh model, which means we can combine it into a bokeh layout just like any other bokeh model:

```python
layout = layout([
    [plot.state],
    [slider, button],
], sizing_mode='fixed')

curdoc().add_root(layout)
```

In [ ]:
# Advanced Exercise: Add a histogram to the bokeh layout next to the datashaded plot
# Hint: Declare the histogram like this: hv.operation.histogram(aggregated, bin_range=(0, 20))
#       then use renderer.get_plot and hist_plot.state and add it to the layout

# Onwards

Although the code above is more complex than in previous sections, it's actually providing a huge range of custom types of interactivity, which if implemented in Bokeh alone would have required far more than a notebook cell of code.  Hopefully it is clear that arbitrarily complex collections of visualizations and interactive controls can be built from the components provided by HoloViews, allowing you to make simple analyses very easily and making it practical to make even quite complex apps when needed.  The [user guide](http://holoviews.org/user_guide), [gallery](http://holoviews.org/gallery/index.html), and [reference gallery](http://holoviews.org/reference) should have all the information you need to get started with all this power on your own datasets and tasks.  Good luck!